OHE implementaions

In [1]:
# import pandas as pd
# import random

# # Define sample text data and labels
# text_samples = [
#     "I love this product! It's amazing.",
#     "Terrible experience, I hate it.",
#     "This is the best purchase I've made.",
#     "Worst service ever. Do not recommend!",
#     "I am very happy with my order.",
#     "So disappointing, not what I expected.",
#     "Absolutely fantastic! Great quality.",
#     "Poor quality, broke in a week.",
#     "Customer support was very helpful.",
#     "I will never buy from here again!"
# ]

# # Generate a random dataset
# random.seed(42)
# dataset_size = 100  # Change this for larger datasets

# data = {
#     "text": [random.choice(text_samples) for _ in range(dataset_size)],
#     "label": [random.choice([0, 1]) for _ in range(dataset_size)]  # 0 = Negative, 1 = Positive
# }

# df = pd.DataFrame(data)

# # Save to CSV
# csv_filename = "random_nlp_dataset.csv"
# df.to_csv(csv_filename, index=False)

# print(f"Dataset saved as {csv_filename}")
# print(df.head())


In [2]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from textblob import TextBlob
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download("averaged_perceptron_tagger")
nltk.download('punkt', download_dir='/usr/local/nltk_data')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /usr/local/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]

True

In [3]:
df=pd.read_csv("random_nlp_dataset.csv")

In [4]:
df.head()

,text,label
0,"Terrible experience, I hate it.",0
1,I love this product! It's amazing.,1
2,I am very happy with my order.,1
3,Worst service ever. Do not recommend!,1
4,Worst service ever. Do not recommend!,1


Stop word Removing

In [5]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove special characters, numbers, and punctuation
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Tokenization
    tokens = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word,pos='v') for word in tokens]
    
    # Join tokens back into a single string
    return ' '.join(tokens)


In [6]:
df['cleaned_text'] = df['text'].apply(preprocess_text)

In [7]:
data_cleaned=df['cleaned_text']

In [8]:
# Bag of Word
vectorizer = CountVectorizer()
bow_features = vectorizer.fit_transform(data_cleaned)

# Convert to DataFrame
bow_df = pd.DataFrame(bow_features.toarray(), columns=vectorizer.get_feature_names_out())
print(bow_df)


    absolutely  amaze  best  break  buy  customer  disappoint  ever  expect  \
0            0      0     0      0    0         0           0     0       0   
1            0      1     0      0    0         0           0     0       0   
2            0      0     0      0    0         0           0     0       0   
3            0      0     0      0    0         0           0     1       0   
4            0      0     0      0    0         0           0     1       0   
..         ...    ...   ...    ...  ...       ...         ...   ...     ...   
95           1      0     0      0    0         0           0     0       0   
96           0      0     0      1    0         0           0     0       0   
97           0      0     1      0    0         0           0     0       0   
98           0      0     0      0    0         0           0     0       0   
99           0      0     1      0    0         0           0     0       0   

    experience  ...  poor  product  purchase  quali

In [9]:
# tf-idf
tfidf_vectorizer = TfidfVectorizer()
tfidf_features = tfidf_vectorizer.fit_transform(data_cleaned)

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_features.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
print(tfidf_df)


    absolutely    amaze  best     break  buy  customer  disappoint  ever  \
0      0.00000  0.00000   0.0  0.000000  0.0       0.0         0.0   0.0   
1      0.00000  0.57735   0.0  0.000000  0.0       0.0         0.0   0.0   
2      0.00000  0.00000   0.0  0.000000  0.0       0.0         0.0   0.0   
3      0.00000  0.00000   0.0  0.000000  0.0       0.0         0.0   0.5   
4      0.00000  0.00000   0.0  0.000000  0.0       0.0         0.0   0.5   
..         ...      ...   ...       ...  ...       ...         ...   ...   
95     0.51734  0.00000   0.0  0.000000  0.0       0.0         0.0   0.0   
96     0.00000  0.00000   0.0  0.527035  0.0       0.0         0.0   0.0   
97     0.00000  0.00000   0.5  0.000000  0.0       0.0         0.0   0.0   
98     0.00000  0.00000   0.0  0.000000  0.0       0.0         0.0   0.0   
99     0.00000  0.00000   0.5  0.000000  0.0       0.0         0.0   0.0   

    expect  experience  ...      poor  product  purchase   quality  recommend  \
0     

In [10]:
# N-grams
vectorizer_ngram = CountVectorizer(ngram_range=(2, 3))
ngram_features = vectorizer_ngram.fit_transform(data_cleaned)

# Convert to DataFrame
ngram_df = pd.DataFrame(ngram_features.toarray(), columns=vectorizer_ngram.get_feature_names_out())
print(ngram_df)


    absolutely fantastic  absolutely fantastic great  best purchase  \
0                      0                           0              0   
1                      0                           0              0   
2                      0                           0              0   
3                      0                           0              0   
4                      0                           0              0   
..                   ...                         ...            ...   
95                     1                           1              0   
96                     0                           0              0   
97                     0                           0              1   
98                     0                           0              0   
99                     0                           0              1   

    best purchase ive  break week  customer support  customer support helpful  \
0                   0           0                 0               

In [11]:
# part of speech
def get_pos_tags(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    return pos_tags

data_pos = data_cleaned.apply(get_pos_tags)
print(data_pos)


0        [(terrible, JJ), (experience, NN), (hate, NN)]
1              [(love, VB), (product, NN), (amaze, NN)]
2                            [(happy, JJ), (order, NN)]
3     [(worst, JJS), (service, NN), (ever, RB), (rec...
4     [(worst, JJS), (service, NN), (ever, RB), (rec...
                            ...                        
95    [(absolutely, RB), (fantastic, JJ), (great, JJ...
96    [(poor, JJ), (quality, NN), (break, NN), (week...
97    [(best, JJS), (purchase, NN), (ive, JJ), (make...
98                           [(happy, JJ), (order, NN)]
99    [(best, JJS), (purchase, NN), (ive, JJ), (make...
Name: cleaned_text, Length: 100, dtype: object


In [12]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['cleaned_text'])
y = df['label']

# Split data into train & test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [13]:
nb_model = MultinomialNB()
nb_model.fit(X_train, y_train)

# Predictions
y_pred_nb = nb_model.predict(X_test)

# Evaluate the model
print("Naïve Bayes Accuracy:", accuracy_score(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))


Naïve Bayes Accuracy: 0.5
              precision    recall  f1-score   support

           0       0.73      0.53      0.62        15
           1       0.22      0.40      0.29         5

    accuracy                           0.50        20
   macro avg       0.47      0.47      0.45        20
weighted avg       0.60      0.50      0.53        20



In [14]:

p=tfidf.transform(['terrible experience hate'])


In [15]:
nb_model.predict(p)


array([0], dtype=int64)